In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marketing-strategy-personalised-offer/sample.csv
/kaggle/input/marketing-strategy-personalised-offer/train_data.csv
/kaggle/input/marketing-strategy-personalised-offer/test_data.csv


In [2]:
# common imports
import numpy as np
import pandas as pd
import os
import io
import warnings

# sklearn specific imports
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import hinge_loss
from sklearn.metrics import f1_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, precision_recall_curve
from sklearn.metrics import precision_score, recall_score, classification_report
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate, cross_val_predict, GridSearchCV, train_test_split
from pprint import pprint

import matplotlib.pyplot as plt
import seaborn as sns

### Data loading

In [3]:
# df = pd.read_csv('../input/marketing-strategy-personalised-offer/sample.csv')

df_train = pd.read_csv("../input/marketing-strategy-personalised-offer/train_data.csv")
df_train.head()

,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,...,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food,Offer Accepted
0,2days,₹100000 or More,4~8,1,less1,Married partner,4 star restaurant,36,0,0,...,0,0,4,67,less1,22,Spring,Location B,0,No
1,2days,₹87500 - ₹99999,4~8,0,4~8,Married partner,Take-away restaurant,50plus,0,0,...,0,1,3,89,1~3,18,Summer,Location B,0,Yes
2,2days,₹87500 - ₹99999,less1,1,1~3,Single,Cold drinks,26,1,0,...,1,1,4,67,less1,7,Winter,Location A,1,Yes
3,10hours,₹37500 - ₹49999,less1,0,1~3,Single,Take-away restaurant,46,1,0,...,0,1,3,89,1~3,7,Summer,Location C,0,No
4,2days,₹100000 or More,never,1,1~3,Single,4 star restaurant,21,0,1,...,1,0,3,40,less1,7,Summer,Location C,0,No


In [4]:
X = df_train.drop("Offer Accepted", axis = 1)
X.head()

,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,...,Job/Job Industry,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food
0,2days,₹100000 or More,4~8,1,less1,Married partner,4 star restaurant,36,0,0,...,Unemployed,0,0,4,67,less1,22,Spring,Location B,0
1,2days,₹87500 - ₹99999,4~8,0,4~8,Married partner,Take-away restaurant,50plus,0,0,...,Unemployed,0,1,3,89,1~3,18,Summer,Location B,0
2,2days,₹87500 - ₹99999,less1,1,1~3,Single,Cold drinks,26,1,0,...,Arts Design Entertainment Sports & Media,1,1,4,67,less1,7,Winter,Location A,1
3,10hours,₹37500 - ₹49999,less1,0,1~3,Single,Take-away restaurant,46,1,0,...,Sales & Related,0,1,3,89,1~3,7,Summer,Location C,0
4,2days,₹100000 or More,never,1,1~3,Single,4 star restaurant,21,0,1,...,Sales & Related,1,0,3,40,less1,7,Summer,Location C,0


In [5]:
y = df_train["Offer Accepted"]
y.head()

0     No
1    Yes
2    Yes
3     No
4     No
Name: Offer Accepted, dtype: object

In [6]:
X.shape, y.shape

((12379, 30), (12379,))

In [7]:
X.isnull().sum()

offer expiration                            0
income_range                                0
no_visited_Cold drinks                    198
travelled_more_than_15mins_for_offer        0
Restaur_spend_less_than20                 121
Marital Status                              0
restaurant type                             0
age                                         0
Prefer western over chinese                 0
travelled_more_than_25mins_for_offer        0
travelled_more_than_5mins_for_offer         0
no_visited_bars                            93
gender                                      0
car                                     12268
restuarant_same_direction_house             0
Cooks regularly                             0
Customer type                               0
Qualification                               0
is foodie                                   0
no_Take-aways                             144
Job/Job Industry                            0
restuarant_opposite_direction_hous

In [8]:
names = list(X.columns)
len(names)

30

### Data Preprocessing

In [9]:
pipe = Pipeline([
    ('imputer', SimpleImputer(strategy = "most_frequent")),
    ('encoder', OrdinalEncoder())
])
pipe

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', OrdinalEncoder())])

In [10]:
X_new = pipe.fit_transform(X)

X = pd.DataFrame(X_new, columns = names)
X.head()

,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,...,Job/Job Industry,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food
0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,3.0,0.0,0.0,...,24.0,0.0,0.0,4.0,1.0,3.0,4.0,0.0,1.0,0.0
1,1.0,8.0,1.0,0.0,1.0,1.0,4.0,6.0,0.0,0.0,...,24.0,0.0,1.0,3.0,2.0,0.0,3.0,1.0,1.0,0.0
2,1.0,8.0,3.0,1.0,0.0,2.0,2.0,1.0,1.0,0.0,...,1.0,1.0,1.0,4.0,1.0,3.0,0.0,2.0,0.0,1.0
3,0.0,4.0,3.0,0.0,0.0,2.0,4.0,5.0,1.0,0.0,...,21.0,0.0,1.0,3.0,2.0,0.0,0.0,1.0,2.0,0.0
4,1.0,1.0,4.0,1.0,0.0,2.0,1.0,0.0,0.0,1.0,...,21.0,1.0,0.0,3.0,0.0,3.0,0.0,1.0,2.0,0.0


In [11]:
y.replace("Yes", 1, inplace = True)
y.replace("No", 0, inplace = True)

y

0        0
1        1
2        1
3        0
4        0
        ..
12374    1
12375    1
12376    0
12377    1
12378    1
Name: Offer Accepted, Length: 12379, dtype: int64

In [12]:
X_train = X
y_train = y

### Decision Tree Classifier model

#### Finding the best parameters using GridSearchCV

In [13]:
from sklearn.tree import DecisionTreeClassifier

In [14]:
tuned_parameters = [{"max_depth": [1, 2, 3, 4, 5, 6, 7, 8, 9],
                     "min_samples_split": [2, 4, 6, 8, 10]}]



clf_CV = GridSearchCV(DecisionTreeClassifier(random_state = 42),
                      tuned_parameters,
                      scoring = 'f1_macro')

clf_CV.fit(X_train, y_train)

print("Best parameters:")
print()
result = clf_CV.best_params_
print(result)
print()

Best parameters:

{'max_depth': 8, 'min_samples_split': 10}



In [15]:
tuned_clf = DecisionTreeClassifier(max_depth = result['max_depth'], 
                                   min_samples_split = result['min_samples_split'], random_state = 42)

In [16]:
tuned_clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=8, min_samples_split=10, random_state=42)

### On test data

In [17]:
df_public_test = pd.read_csv("../input/marketing-strategy-personalised-offer/test_data.csv")

X_test_public = pipe.fit_transform(df_public_test)

X_test_public = pd.DataFrame(X_test_public, columns = names)
print(X_test_public.shape)
X_test_public.head()

(5305, 30)


,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,...,Job/Job Industry,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food
0,1.0,4.0,0.0,1.0,0.0,0.0,4.0,6.0,0.0,1.0,...,21.0,1.0,1.0,4.0,2.0,3.0,0.0,1.0,2.0,0.0
1,0.0,3.0,1.0,0.0,0.0,3.0,2.0,0.0,1.0,0.0,...,5.0,1.0,1.0,0.0,1.0,3.0,3.0,1.0,0.0,1.0
2,1.0,1.0,4.0,1.0,4.0,2.0,2.0,0.0,1.0,0.0,...,24.0,1.0,0.0,4.0,1.0,4.0,0.0,1.0,2.0,0.0
3,1.0,3.0,4.0,1.0,0.0,1.0,3.0,4.0,0.0,0.0,...,24.0,1.0,1.0,0.0,0.0,3.0,3.0,0.0,2.0,1.0
4,0.0,2.0,4.0,0.0,0.0,2.0,0.0,2.0,1.0,0.0,...,24.0,0.0,0.0,0.0,2.0,0.0,3.0,1.0,1.0,1.0


In [18]:
y_pred_public = tuned_clf.predict(X_test_public)
print(np.unique(y_pred_public))
print(y_pred_public.shape)

[0 1]
(5305,)


### Output

In [19]:
sub = pd.DataFrame(y_pred_public, columns = ["Offer Accepted"])
sub.insert(0, 'id', range(0, len(sub)))
sub['Offer Accepted'].replace(1, "Yes", inplace = True)
sub['Offer Accepted'].replace(0, "No", inplace = True)

In [20]:
sub.columns

Index(['id', 'Offer Accepted'], dtype='object')

In [21]:
sub.to_csv('/kaggle/working/submission.csv', index = False)